In [1]:
# import random as rd
# rd.seed(0)

In [2]:
#import libraries
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import xgboost as xg
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('data2.csv')

In [4]:
data['dist'] = np.linalg.norm(data.loc[:, ["x378","y378"]].values, axis=1)
features = data.drop(["file_name","pitch","yaw","roll","h","v","t","scale","dist"], axis=1)
labels = data[["pitch","yaw","roll"]]
features.loc[:,:]=features.loc[:,:].div(data["dist"],axis=0)

In [5]:
scaler= MinMaxScaler()
pca=PCA(n_components=0.98)  

In [20]:
# remark: we fix random_state the end, to make sure we always get the same results
features_train, features_validation, labels_train, labels_validation = train_test_split(
    features, labels, test_size=0.2, random_state=3)

In [21]:
rf_yaw =RandomForestRegressor(random_state=42)
PIPE_LINE_yaw =make_pipeline(scaler,pca,rf_yaw).fit(features_train, labels_train["yaw"])
print("RandomForestRegressor yaw train:",PIPE_LINE_yaw.score(features_train, labels_train["yaw"]))
print("RandomForestRegressor yaw test:", PIPE_LINE_yaw.score(features_validation, labels_validation["yaw"]))

RandomForestRegressor yaw train: 0.962317437971364
RandomForestRegressor yaw test: 0.88393683312462


In [22]:
rf_pitch =RandomForestRegressor(random_state=42)
PIPE_LINE_pitch =make_pipeline(scaler,pca,rf_pitch).fit(features_train, labels_train["pitch"])
print("RandomForestRegressor pitch train:",PIPE_LINE_pitch.score(features_train, labels_train["pitch"]))
print("RandomForestRegressor pitch test:", PIPE_LINE_pitch.score(features_validation, labels_validation["pitch"]))

RandomForestRegressor pitch train: 0.7689562575711393
RandomForestRegressor pitch test: 0.6324876311150796


In [23]:
rf_roll =RandomForestRegressor(random_state=42)
PIPE_LINE_roll =make_pipeline(scaler,pca,rf_roll).fit(features_train, labels_train["roll"])
print("RandomForestRegressor yaw train:",PIPE_LINE_roll.score(features_train, labels_train["roll"]))
print("RandomForestRegressor yaw test:", PIPE_LINE_roll.score(features_validation, labels_validation["roll"]))

RandomForestRegressor yaw train: 0.8826002588173055
RandomForestRegressor yaw test: 0.792197557115155


In [25]:
filename = 'yaw.sav'
pickle.dump(PIPE_LINE_yaw, open(filename, 'wb'))
filename = 'pitch.sav'
pickle.dump(PIPE_LINE_pitch, open(filename, 'wb'))
filename = 'roll.sav'
pickle.dump(PIPE_LINE_roll, open(filename, 'wb'))